In [1]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Block 1: Import necessary libraries
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx
from sklearn.model_selection import train_test_split
import networkx as nx
from objects.user import UserObject
from pprint import pprint 
from utils.feature_extractor import FeatureExtractor
from utils.config_manager import ConfigManager
from data_set_managers.json_dataset_manager import JsonDatasetManager
from objects.timeline import TimelineObject
import json 
import pickle
from datetime import datetime, timedelta
import numpy as np

C:\Users\JS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\JS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages
No model chosen, model unsloth/Llama-3.1-Storm-8B-bnb-4bit selected.


C:\Users\JS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Block 2: Load data
print("Loading telegram post data...")
telegram_post_data = JsonDatasetManager(str(ConfigManager().location_of_post_data))

print("Loading ground truth extremist post data...")
ground_truth_extremist_post_data = JsonDatasetManager(ConfigManager().location_of_ground_truth_extremist_post_data)

# Initialize feature extractor
print("Initializing FeatureExtractor...")
feature_extractor = FeatureExtractor(
    telegram_post_data.get_list_of_posts(), 
    ground_truth_extremist_post_data.get_list_of_posts()
)

Loading telegram post data...
Loading ground truth extremist post data...
Initializing FeatureExtractor...
No model chosen, model unsloth/Llama-3.1-Storm-8B-bnb-4bit selected.


c:\Users\JS\Documents\tools\Tribal\Tribal\utils\feature_extractor.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  embeddings = torch.tensor(embeddings, dtype=torch.float)


Epoch 1, Loss: 0.005771621507711263
Epoch 2, Loss: 6.629083414521009e-06
Epoch 3, Loss: 1.174403536726617e-06
Epoch 4, Loss: 2.681721007951826e-07
Epoch 5, Loss: 6.605760283702735e-08


In [4]:
# Block 3: Prepare user data and graph structure
users = telegram_post_data.get_all_user_data()

# Build the graph based on users' connections and features
graph = nx.Graph()

# Initialize mappings for themes, operational status, and roles
theme_to_num = {}
num_to_theme = {}

ops_to_num = {}
num_to_ops = {}

role_to_num = {}
num_to_role = {}

node_labels = []  # Placeholder for node labels
node_features = []  # Placeholder for feature vectors

CHOSEN_OVERALL_WINDOW_START, CHOSEN_OVERALL_WINDOW_END = telegram_post_data.get_timeframe()
posts = telegram_post_data.get_all_user_data(
    start_time=CHOSEN_OVERALL_WINDOW_START, end_time=CHOSEN_OVERALL_WINDOW_END
)


In [5]:
import pickle
import os

# Define the pickle file name
pickle_file = 'window.pkl'

# Try to load the pickled window if it exists
if os.path.exists(pickle_file):
    with open(pickle_file, 'rb') as f:
        window = pickle.load(f)
    print("Loaded the pickled window.")
else:
    print("Pickle file does not exist. Creating a new window.")

    # Assuming 'timeline' object and 'posts' are already defined
    timeline = TimelineObject(feature_extractor)
    timeline.posts = posts
    timeline.make_new_window(CHOSEN_OVERALL_WINDOW_START, CHOSEN_OVERALL_WINDOW_END)

    # Get the window object
    window = timeline.windows[0]

    # Save the window object to a pickle file
    with open(pickle_file, 'wb') as f:
        pickle.dump(window, f)
    print("Created and pickled a new window.")


Pickle file does not exist. Creating a new window.


C:\Users\JS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\quantizers\auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
C:\Users\JS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{ "is_operational_planning": "no", "reasoning": "The post does not contain any keywords or phrases related to violent action, indicators of seeking knowledge or skills related to carrying out violence, discussion about logistics or coordination with others, or leakage of intentions or plans. The tone and emotional intensity of the post do not suggest an individual close to taking action. The content does not align with the research paper's findings on online signals of extremist mobilization." }
{ "theme": "Travel", "reasoning": "The post talks about visiting the beach, which is a common tourist destination, and enjoying sunshine, indicating a trip or vacation." }
{ "is_operational_planning": "no", "reasoning": "The post does not contain any keywords or phrases related to violent action, nor does it indicate the user is seeking to acquire knowledge or skills related to carrying out violence. There is no discussion about logistics, timelines, specific targets, or coordination with other

TypeError: The schema argument must be a Pydantic model class.

In [5]:
# Block 4: Extract user features and populate the graph
current_ops_index = 0
current_theme_index = 0
current_role_index = 0

# Dictionaries to map operational states, themes, and roles to numerical values
ops_to_num = {}
num_to_ops = {}

theme_to_num = {}
num_to_theme = {}

role_to_num = {}
num_to_role = {}

# Define the fixed set of POS tags and emotions based on your dataset
pos_tags = ['NN', 'VB', 'JJ', 'RB']  # Noun, Verb, Adjective, Adverb (example)
emotions = ['anger', 'fear', 'joy', 'sadness', 'surprise', 'disgust']  

node_labels = []  # Placeholder for node labels
node_features = []  # Placeholder for feature vectors

for user in window.users.values():
    user_data = user.get_dict()
    centrality = user_data["centrality"]
    avrg_sentiment = user_data["avrg_sentiment"]
    avrg_toxicity = user_data["avrg_toxicity"]
    operational = user_data["avrg_is_operational"]
    extremism = user_data["extremism"]
    role = user_data["role"]
    avrg_capital_letter_word_frequency = user_data["avrg_capital_letter_word_frequency"]
    avrg_emotion_scores = user_data["avrg_emotion_scores"]
    average_hate_speech_lexicon_counts = user_data["average_hate_speech_lexicon_counts"]
    avrg_text_vector = user_data["avrg_text_vector"]  # This should be a list or numpy array
    avrg_pos_counts = user_data["avrg_pos_counts"]  # Dictionary of POS counts

    # Handling operational mapping
    if operational not in ops_to_num:
        ops_to_num[operational] = current_ops_index
        num_to_ops[current_ops_index] = operational
        current_ops_index += 1
    operational_num = ops_to_num[operational]

    # Handling theme mapping
    theme = user_data["avrg_theme"]
    if theme not in theme_to_num:
        theme_to_num[theme] = current_theme_index
        num_to_theme[current_theme_index] = theme
        current_theme_index += 1
    avrg_theme_num = theme_to_num[theme]

    # Handling role mapping
    if role not in role_to_num:
        role_to_num[role] = current_role_index
        num_to_role[current_role_index] = role
        current_role_index += 1
    role_num = role_to_num[role]

    # Processing POS counts into a fixed-size vector
    pos_counts_vector = [avrg_pos_counts.get(tag, 0) for tag in pos_tags]

    # Processing emotion scores into a fixed-size vector
    emotion_scores_vector = [avrg_emotion_scores.get(emotion, 0.0) for emotion in emotions]

    # Summing the average hate speech lexicon counts
    average_hate_speech_count = sum(average_hate_speech_lexicon_counts.values())

    # Ensure avrg_text_vector is a list (convert if necessary)
    if isinstance(avrg_text_vector, np.ndarray):
        avrg_text_vector = avrg_text_vector.tolist()

    # Create the feature vector by concatenating all features
    feature_vector = [
        centrality,
        avrg_sentiment,
        avrg_toxicity,
        operational_num,
        avrg_theme_num,
        extremism,
        role_num,
        avrg_capital_letter_word_frequency,
        average_hate_speech_count,
    ] + pos_counts_vector + emotion_scores_vector + avrg_text_vector

    # Append the feature vector and label
    node_features.append(feature_vector)
    node_labels.append(extremism)  # Assuming 'extremism' is the label (adjust if necessary)

    # Add the node to the graph
    graph.add_node(user_data["username"], profile=torch.tensor(feature_vector, dtype=torch.float))


NameError: name 'window' is not defined

In [6]:
# Block 5: Extract user features and populate the graph
current_ops_index = 0
current_theme_index = 0
current_role_index = 0

# Dictionaries to map operational states, themes, and roles to numerical values
ops_to_num = {}
num_to_ops = {}

theme_to_num = {}
num_to_theme = {}

role_to_num = {}
num_to_role = {}

for username, user in window.users.items():
    user_data = user.get_dict()
    centrality = user_data.get("centrality", 0)
    avrg_sentiment = user_data.get("avrg_sentiment", 0)
    avrg_toxicity = user_data.get("avrg_toxicity", 0)
    operational = user_data.get("avrg_is_operational", 'unknown')
    extremism = user_data.get("extremism", 0)
    role = user_data.get("role", 'unknown')

    # Handling operational mapping
    if operational not in ops_to_num:
        ops_to_num[operational] = current_ops_index
        num_to_ops[current_ops_index] = operational
        current_ops_index += 1
    operational_num = ops_to_num[operational]

    # Handling theme mapping
    theme = user_data.get("avrg_theme", 'unknown')
    if theme not in theme_to_num:
        theme_to_num[theme] = current_theme_index
        num_to_theme[current_theme_index] = theme
        current_theme_index += 1
    avrg_theme_num = theme_to_num[theme]

    # Handling role mapping
    if role not in role_to_num:
        role_to_num[role] = current_role_index
        num_to_role[current_role_index] = role
        current_role_index += 1
    role_num = role_to_num[role]

    # Create feature vector: centrality, sentiment, toxicity, operational, theme, extremism, role
    feature_vector = [
        centrality, 
        avrg_sentiment, 
        avrg_toxicity, 
        operational_num, 
        avrg_theme_num, 
        extremism, 
        role_num
    ]

    # Append the feature vector and label
    node_features.append(torch.tensor(feature_vector, dtype=torch.float))
    node_labels.append(extremism)

    # Add the node to the graph
    graph.add_node(username, profile=torch.tensor(feature_vector, dtype=torch.float))

    # Add edges based on connections (assuming user_data contains 'connections')
    connections = user_data.get('connections', [])
    for connected_user in connections:
        graph.add_edge(username, connected_user)


In [7]:
# Block 6: Convert NetworkX graph to PyTorch Geometric format
from torch_geometric.utils import from_networkx

data = from_networkx(graph)

# Update node features and labels
data.x = torch.stack([graph.nodes[node]['profile'] for node in graph.nodes])
data.y = torch.tensor(node_labels, dtype=torch.long)  # Labels for each node (extremist or non-extremist)


In [8]:
# Block 7: Split data into train, validation, and test sets
train_mask_indices, val_test_mask_indices = train_test_split(
    range(data.num_nodes), test_size=0.4, random_state=42
)
val_mask_indices, test_mask_indices = train_test_split(
    val_test_mask_indices, test_size=0.5, random_state=42
)

# Create mask tensors for train, validation, and test sets
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[train_mask_indices] = True

val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
val_mask[val_mask_indices] = True

test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask[test_mask_indices] = True

# Assign masks to data
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask


In [9]:
# Block 8: Define GCN model for node classification
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [10]:
# Block 9: Initialize model, optimizer, and loss function
input_dim = data.x.shape[1]  # Updated input dimension based on new feature vector size
hidden_dim = 64  # You can adjust this value
output_dim = 2   # Assuming binary classification (extremist or non-extremist)

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GCN(in_channels=input_dim, hidden_channels=hidden_dim, out_channels=output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


In [11]:
# Block 10: Function to evaluate model accuracy
def evaluate(mask):
    model.eval()
    with torch.no_grad():
        out = model(data)  # Forward pass
        pred = out.argmax(dim=1)  # Get predicted classes
        correct = pred[mask].eq(data.y[mask]).sum().item()  # Count correct predictions
        accuracy = correct / mask.sum().item()  # Compute accuracy
    return accuracy


In [12]:
# Block 11: Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        train_acc = evaluate(data.train_mask)
        val_acc = evaluate(data.val_mask)
        print(
            f'Epoch {epoch}, Loss: {loss.item():.4f}, '
            f'Train Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}'
        )


In [13]:
# Block 12: Evaluate model on the test set
test_acc = evaluate(data.test_mask)
print(f'Test Accuracy: {test_acc:.4f}')


In [14]:
# Block 13: Function to predict the effect of adding a user
def predict_add_user_gnn(new_user_id, new_user_features, connections, graph, model):
    print(f"Predicting changes by adding user {new_user_id}...")

    # Add the new user node and connections
    graph.add_node(
        new_user_id, 
        profile=torch.tensor(new_user_features, dtype=torch.float)
    )
    for connection in connections:
        graph.add_edge(new_user_id, connection)

    # Convert updated graph to PyTorch Geometric format
    data = from_networkx(graph)
    data.x = torch.stack([graph.nodes[node]['profile'] for node in graph.nodes])
    data.edge_index = torch.tensor(
        list(graph.edges), dtype=torch.long
    ).t().contiguous()

    # Predict using the GCN
    model.eval()
    with torch.no_grad():
        out = model(data)

    predicted_features = out[-1]  # Assuming the new user is the last node
    print(f"Predicted features for new user {new_user_id}: {predicted_features}")
    return predicted_features


In [15]:
# Block 14: Function to predict the effect of removing a user
def predict_remove_user_gnn(user_id, graph, model):
    print(f"Predicting changes by removing user {user_id}...")

    if graph.has_node(user_id):
        graph.remove_node(user_id)
    else:
        print(f"User {user_id} does not exist in the graph.")
        return None

    data = from_networkx(graph)
    data.x = torch.stack([graph.nodes[node]['profile'] for node in graph.nodes])
    data.edge_index = torch.tensor(
        list(graph.edges), dtype=torch.long
    ).t().contiguous()

    model.eval()
    with torch.no_grad():
        out = model(data)

    print(
        f"Predicted features for the remaining nodes after removing user {user_id}: {out}"
    )
    return out


In [16]:
# Block 15: Example usage of prediction functions
new_user_id = 'new_user'
new_user_features = [
    0.6, 0.5, 0.4, 
    ops_to_num.get('unknown', 0), 
    theme_to_num.get('unknown', 0), 
    0,  # extremism
    role_to_num.get('unknown', 0)
]  # Example features
connections = ['some_user', 'another_user']  # Example connections to other users

# Predict effect of adding a user
predicted_features = predict_add_user_gnn(
    new_user_id, new_user_features, connections, graph, model
)

# Predict effect of removing a user
removed_user_predictions = predict_remove_user_gnn('some_user', graph, model)


In [17]:
# Block 16: Function to predict missing features after removing and re-adding a user
def predict_removed_user_features(user_id, graph, model):
    if not graph.has_node(user_id):
        print(f"User {user_id} does not exist in the graph.")
        return None

    print(f"Removing user {user_id} from the graph...")
    user_profile = graph.nodes[user_id]['profile']
    graph.remove_node(user_id)

    incomplete_profile = user_profile.clone()
    incomplete_profile[1:3] = float('nan')
    print(
        f"Stripped features (sentiment and toxicity) for user {user_id}: {incomplete_profile}"
    )

    print(f"Re-adding user {user_id} with missing features...")
    graph.add_node(user_id, profile=incomplete_profile)

    original_connections = users[user_id].get('connections', [])
    for connection in original_connections:
        graph.add_edge(user_id, connection)

    data = from_networkx(graph)
    data.x = torch.stack([graph.nodes[node]['profile'] for node in graph.nodes])
    data.edge_index = torch.tensor(
        list(graph.edges), dtype=torch.long
    ).t().contiguous()

    model.eval()
    with torch.no_grad():
        predicted_features = model(data)

    original_features = user_profile
    node_index = list(graph.nodes).index(user_id)
    predicted_features_for_user = predicted_features[node_index]
    print(f"Original features: {original_features}")
    print(f"Predicted features: {predicted_features_for_user}")

    return original_features, predicted_features_for_user


In [18]:
# Block 17: Example usage of removing a user and predicting missing features
user_to_remove = 'some_user'  # Specify the user ID to remove and test
result = predict_removed_user_features(user_to_remove, graph, model)

if result:
    original, predicted = result
    # Calculate the accuracy of the prediction for the missing features
    missing_indices = [1, 2]  # Indices of the stripped features (sentiment and toxicity)
    error = torch.abs(original[missing_indices] - predicted[missing_indices])
    print(f"Prediction error for missing features: {error}")


In [19]:
# Block 18: Compute and plot correlation coefficients between features and "extremism" labels

import matplotlib.pyplot as plt

def calculate_correlation(data):
    # Extract features (node features) and labels (extremism labels)
    features = data.x  # This is the matrix of node features
    labels = data.y  # This is the "extremism" label (0 or 1)

    # Convert features and labels to numpy arrays for correlation calculation
    features_np = features.numpy()
    labels_np = labels.numpy()

    # Calculate the correlation coefficient for each feature with respect to the labels
    correlations = []
    for i in range(features_np.shape[1]):
        feature_column = features_np[:, i]
        correlation = np.corrcoef(feature_column, labels_np)[0, 1]
        correlations.append(correlation)

    return correlations

# Calculate the correlation coefficients
correlations = calculate_correlation(data)

# Plotting the correlation coefficients as a bar graph
feature_names = [
    'Centrality', 'Sentiment', 'Toxicity', 'Operational Status', 
    'Theme', 'Extremism', 'Role'
]  # Adjust feature names accordingly
plt.figure(figsize=(10, 6))
plt.bar(range(len(correlations)), correlations, tick_label=feature_names)
plt.xlabel('Features')
plt.ylabel('Correlation with Extremism')
plt.title('Correlation between Features and Extremism')
plt.xticks(rotation=45)
plt.tight_layout()

# Display the plot
plt.show()
